In [ ]:
import os
import dash
from dash import dcc
from dash import html
import dash_bootstrap_components as dbc
import plotly.express as px
import plotly.graph_objects as go
from dash.dependencies import Input, Output, State
import tensorflow as tf
from tensorflow.python.keras.models import load_model

import matplotlib.image as mpimg
import matplotlib.pylab as plt

import pandas as pd
import numpy as np

In [ ]:
# Get absolute path of repository
path_repo = os.path.dirname(os.path.abspath('app'))

# Load models
path_model_150 = os.path.join(path_repo, 'models', 'MobileNetV3L_Kaggle_p150_e50')
path_model_300 = os.path.join(path_repo, 'models', 'MobileNetV3L_Kaggle_p300_e50')
# model_150 = tf.keras.models.load_model(path_model_150)
model_150 = load_model(path_model_150)
#model_300 = load_model(path_model_300)

In [ ]:
IMG_HEIGHT = 150

In [ ]:
# Creating a function that preprocess the custom data
def load_and_prep_image(filename, img_shape = IMG_HEIGHT):
  img = tf.io.read_file(filename) #read image
  img_original = mpimg.imread(filename)
  img = tf.image.decode_image(img) # decode the image to a tensor
  img = tf.image.resize(img, size = [img_shape, img_shape]) # resize the image
  return img, img_original

In [ ]:
def pred_and_plot(model, filename, class_names):
  """
  Imports an image located at filename, makes a prediction on it with
  a trained model and plots the image with the predicted class as the title.
  """
  # Import the target image and preprocess it
  img, img_original = load_and_prep_image(filename)

  # Make a prediction
  pred = model.predict(tf.expand_dims(img, axis=0))

  # Get the predicted class
  if len(pred[0]) > 1: # check for multi-class
    pred_class = class_names[pred.argmax()] # if more than one output, take the max
  else:
    pred_class = class_names[int(tf.round(pred)[0][0])] # if only one output, round

  # Plot the image and predicted class
  plt.imshow(img_original)
  plt.title(f"Prediction: {pred_class}")
  plt.axis(False);

In [ ]:
class_names = ['BLACK REDSTART', 'BLACKBIRD', 'COAL TIT', 'COLLARED DOVE', 'COMMON CHAFFINCH', 'COMMON REDSTART', 'EURASIEN BLUE TIT', 'EURASIEN JAY', 'EURASIEN TREE SPARROW', 'EUROPEAN GREENFINCH', 'EUROPEAN JACKDAW', 'EUROPEAN SERIN', 'GARDEN WARBLER', 'GREAT SPOTTED WOODPECKER', 'LONG TAILED TIT', 'ROOK', 'SHORT-TOED TREECREEPER', 'SONG THRUSH', 'SPOTTED FLYCATCHER', 'WHITE WAGTAIL', 'WILLOW WARBLER', 'YELLOWHAMMER']

In [ ]:
amsel = path_repo + '/images_test-samples/' + 'Amsel.jpeg'

In [ ]:
# amsel - blackbird
pred_and_plot(model_150,amsel, class_names)

---

In [ ]:
df = pd.read_csv(os.path.join(path_repo, 'labelmap_europe.txt'), header=None, names=['species'])

In [ ]:
labelmap = df.species.tolist()
labelmap

---

In [ ]:
def predict(model, label, image):
    '''
    image: path to image
    label: labelmap (as list)
    model: the model
    '''
    if model==model_150:  # or 150x150 MobileNetV3L ?
        IMG_SHAPE = 150
    elif model==model_300:
        IMG_SHAPE = 300

    # preprocessing
    img = tf.io.read_file(image) #read image
    img = tf.image.decode_image(img) # decode the image to a tensor
    img = tf.image.resize(img, size = [IMG_SHAPE, IMG_SHAPE]) # resize the image

    # prediction
    pred = model.predict(tf.expand_dims(img, axis=0))

    # get 3 indices with the highest values and sort descending
    ind = np.argpartition(pred.flatten(), -3)[-3:]
    top3 = pred.flatten()[ind]

    # create list of tuples with indices and values
    top = []
    for idx, val in zip(ind, top3):
        top.append((val, idx))

    # per default, it will be sorted by first entry of tuples, but ascending
    top.sort(reverse=True)
    
    #
    values = [top[i][0]*100 for i in range(3)]
    names = [label[top[i][1]] for i in range(3)]
    

    return values, names

In [ ]:
pred_vals, pred_label = predict(model_150, labelmap, amsel)

In [ ]:
def bar_plot(values, names):
    z=[12,24,48] # colors of the viridis colormap
    fig = go.Figure(go.Bar(
            x=values,
            y=names,
            orientation='h',
            marker=dict(color = z,
                        colorscale='viridis')
                        )   
                    )

    # Adding labels
    annotations = []
    y_s = np.round(values, decimals=2)
    
    for yd, xd in zip(y_s, names):
        # labeling the bar
        annotations.append(dict(
                            y=xd, x=yd + 5,
                            text=str(yd) + '%',
                            font=dict(family='Arial', size=16,
                                      #color='rgb(50, 171, 96)'
                                      ),
                            showarrow=False
                            ))
    fig.update_layout(annotations=annotations)
    fig.update_xaxes(visible=False, showticklabels=False)
    #fig.show()

    return fig

In [ ]:
bar_plot(pred_vals, pred_label)